# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [9]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [10]:
input_keyword = "電子紙"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1:1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

電子紙: %E9%9B%BB%E5%AD%90%E7%B4%99
/wiki/%E9%9B%BB%E5%AD%90%E7%B4%99


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [11]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph.get_text())


電子紙，簡稱ePaper，是顯示器技術，重點在於模仿以在紙上印刷、書寫的視覺觀感，而耗電量極小。不同於一般的平板顯示器以發光達成顯示功能，電子紙如同普通紙一樣，依靠環境光照亮，所以理論上閱讀起來較舒適，而且其顯示的影像能在陽光直照下仍然清晰可見，可視角極廣理論上為180度，其對比度比其他顯示技術高不少[來源請求]，大至而報章印刷效果相同，甚至更好。

大部份電子紙在沒有電源情況下能顯示原先圖片和文字。部份設計以塑料背版和有機電子零件構成柔性電子紙，首款面向歐洲市場的柔性電子紙預計將於2012年4月上市。[來源請求]

彩色電子紙已經商業性投產，能顯示四千種顏色，現階段生產商正向影片播放方向發展，樣機已經能播放每秒30格的黑白影片。

電子紙顯示技術現已經應用於零售商店價簽、數位標牌、公車到站時間表、電子看板、手機螢幕、電子書閱讀器、穿戴式電子裝置。

微膠囊電泳顯示器(Microencapsulated EPD, Microencapsulated ElectroPhoretic Display)是一種運用施加電場重新排列帶電色素粒子以顯像的技術。[1]

]]

1990 年代, 另一種建構於「微膠囊電泳顯示」電子墨水的技術原型在MIT出的研究所團隊被實作出並被稱為Nature Paper。J.D. Albert, Barrett Comiskey, Joseph Jacobson, Jeremy Rubin and Russ Wilcox 等人在 1997年將此技術商業化而創立了E Ink公司。 緊接著二年後,E Ink公司和Philips Components建立了夥伴關係以共同研發並行銷此項技術。在2005年，Philips將這項電紙事業和相關專利讓售給Prime View International(PVI) 公司。


長久以來，研發人員企圖創造一種可撓，低價的電紙類顯示介質。在這種情境下，微粒子顯示技術 吸引了 研發人員的長期關注。可切換對比的顯示技術 以電氣驅動 高散射性 或吸收性的微粒子( 0.1-5mm尺度範圍內 )，不同於主流的液晶顯示的微分子特性 . 微粒子型的顯示技術 的雙穩態本質, 可以 直流電場 達到 非常低的耗電效率 同時擁有高對比和光反射率。這些特性, 結合 near-lambertian 觀賞特性，產生 紙上墨跡 的視覺效果。但這

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [12]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    if len(a_tag) > 0:
        for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [顯示器] /wiki/%E6%98%BE%E7%A4%BA%E5%99%A8
外部連結: [紙] /wiki/%E7%B4%99
外部連結: [印刷] /wiki/%E5%8D%B0%E5%88%B7
外部連結: [平板顯示器] /wiki/%E5%B9%B3%E6%9D%BF%E6%98%BE%E7%A4%BA%E5%99%A8
外部連結: [可視角] /wiki/%E9%A1%AF%E7%A4%BA%E5%99%A8%E5%8F%AF%E8%A6%96%E8%A7%92%E5%BA%A6
外部連結: [對比度] /wiki/%E5%B0%8D%E6%AF%94%E5%BA%A6
外部連結: [[來源請求]] /wiki/Wikipedia:%E5%88%97%E6%98%8E%E6%9D%A5%E6%BA%90
外部連結: [[來源請求]] /wiki/Wikipedia:%E5%88%97%E6%98%8E%E6%9D%A5%E6%BA%90
外部連結: [數位標牌] /wiki/%E6%95%B0%E5%AD%97%E6%A0%87%E7%89%8C
外部連結: [電子書閱讀器] /wiki/%E9%9B%BB%E5%AD%90%E6%9B%B8%E9%96%B1%E8%AE%80%E5%99%A8
外部連結: [電場] /wiki/%E9%9B%BB%E5%A0%B4
外部連結: [MIT] /wiki/%E9%BA%BB%E7%9C%81%E7%90%86%E5%B7%A5%E5%AD%A6%E9%99%A2
外部連結: [E Ink公司] /wiki/E_Ink%E5%85%AC%E5%8F%B8
外部連結: [E Ink公司] /wiki/E_Ink%E5%85%AC%E5%8F%B8
外部連結: [LCD] /wiki/LCD
外部連結: [燒屏] /wiki/%E7%A3%B7%E8%B4%A8%E7%83%99%E5%8D%B0


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [15]:
def WikiArticle(key_word_link, key_word, recursive, max_recursive_depth):
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        
        with open('{}.txt'.format(key_word), 'w', encoding='utf-8') as f:
            for paragraph in content:
                f.write(paragraph.get_text())

    
        #
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})

        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            if len(a_tag) > 0:
                for link_string in a_tag:
                    a_link = link_string["href"]       # 外部連結的網址
                    a_keyword = link_string.get_text()  # 外部連結的中文名稱
                    external_link_dict[a_keyword] = a_link

                    
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                WikiArticle(v, k, recursive, max_recursive_depth)   # 再次呼叫同樣的函數，執行同樣的流程
                

### 執行前個步驟定義好的爬蟲主程式

In [ ]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0, max_recursive_depth)

遞迴層[0] - /wiki/%E9%9B%BB%E5%AD%90%E7%B4%99 (電子紙)
遞迴層[1] - /wiki/%E6%98%BE%E7%A4%BA%E5%99%A8 (顯示器)
遞迴層[2] - /wiki/%E9%9B%BB%E8%85%A6 (電腦)
遞迴層[2] - /wiki/%E7%94%B5%E8%A7%86 (電視)
遞迴層[2] - /wiki/%E9%98%B4%E6%9E%81%E5%B0%84%E7%BA%BF%E7%AE%A1 (陰極射線管)
遞迴層[2] - /wiki/Apple_I (Apple I)
遞迴層[2] - /wiki/LCD (LCD)
遞迴層[2] - /wiki/LED (LED)
遞迴層[2] - /wiki/%E8%8B%B1%E5%AF%B8 (英吋)
遞迴層[2] - /wiki/%E9%95%B7%E5%AF%AC%E6%AF%94_(%E5%BD%B1%E5%83%8F) (長寬比)
遞迴層[1] - /wiki/%E7%B4%99 (紙)
遞迴層[2] - /wiki/%E7%BA%A4%E7%BB%B4 (纖維)
遞迴層[2] - /wiki/%E8%A4%87%E5%AF%AB%E7%B4%99 (複寫紙)
遞迴層[2] - /wiki/%E8%A1%9B%E7%94%9F%E7%B4%99 (衛生紙)
遞迴層[2] - /wiki/%E9%9D%A2%E7%B4%99 (面紙)
遞迴層[2] - /wiki/%E4%B8%AD%E5%9C%8B (中國)
遞迴層[2] - /wiki/%E4%BA%BA%E7%B1%BB%E6%96%87%E6%98%8E (人類文明)
遞迴層[2] - /wiki/%E4%B8%9C%E6%B1%89 (東漢)
遞迴層[2] - /wiki/%E8%A8%B1%E6%85%8E (許慎)
遞迴層[2] - /wiki/%E8%AA%AA%E6%96%87%E8%A7%A3%E5%AD%97 (說文解字)
遞迴層[2] - /wiki/%E6%B8%85%E6%9C%9D (清)
遞迴層[2] - /wiki/%E6%AE%B5%E7%8E%89%E8%A3%81 (段玉裁)
遞迴層[2] - /wiki/%E5%8F%A4%E5%9F%83%E5